## HyperLang Sample Concept


In this sample concept, we will generate transcripts in Spanish given the country of origin and scenario. For now, we manually specify the country and scenario in this notebook, but, in the future we can create more
specific parameters (e.g. difficulty, number of speakers in conversation) that will be inputted by the end user.

### Helper Classes

In [11]:
"""
Handles LLM interactions using Hugging Face's API for text generation.
"""
from dotenv import load_dotenv
from huggingface_hub import InferenceClient
from typing import Dict, Any, AsyncGenerator
import asyncio
import config
import os


load_dotenv()


class PromptHandler:
   """
   Class to handle loading and formatting of prompt templates.
   """

   def __init__(self, template_path: str) -> None:
       """Initialize with path to prompt template file."""
       self.template_path = template_path

   def load_prompt(self) -> str:
       """Load prompt template from file."""
       with open(self.template_path, 'r') as file:
           return file.read()

   def format_prompt(self, country_name: str, scenario: str) -> str:
       """
       Format prompt with country name and conversation scenario.

       Args:
           country_name: Country Name
           scenario: Conversation Scenario

       Returns:
           Formatted prompt for LLM
       """
       template = self.load_prompt()
       return template.format(
           country_name=country_name,
           scenario=scenario
       )
   def format_prompt_scenario(self, scenario: str) -> str:
       """
       Format prompt with conversation scenario.

       Args:
           scenario: Conversation Scenario

       Returns:
           Formatted prompt for LLM
       """
       template = self.load_prompt()
       return template.format(scenario=scenario)


class LLMAdapter:
   """Handles LLM interactions for text generation."""

   def __init__(self, model_name: str = "mistralai/Mistral-7B-Instruct-v0.3", temperature: float = 0.7) -> None:
       """Initialize with model configuration."""
       self.client = InferenceClient(api_key=os.getenv('HUGGINGFACE_TOKEN'))
       self.model_name = model_name
       self.temperature = temperature

   def generate_response(self, prompt: str) -> str:
       """
       Generate complete response from LLM.

       Args:
           prompt: Input text for LLM

       Returns:
           Complete generated text
       """
       try:
           messages = [{"role": "user", "content": prompt}]

           completion = self.client.chat.completions.create(
               model=self.model_name,
               messages=messages,
               temperature=self.temperature,
               max_tokens=10000
           )

           return completion.choices[0].message.content

       except Exception as e:
           print(f"Exception occurred: {str(e)}")
           return f"Error: {str(e)}"

   async def generate_response_stream(self, prompt: str) -> AsyncGenerator[str, None]:
       """
       Generate streaming response from LLM.

       Args:
           prompt: Input text for LLM

       Returns:
           AsyncGenerator yielding text chunks
       """
       try:
           messages = [{"role": "user", "content": prompt}]

           stream = self.client.chat.completions.create(
               model=self.model_name,
               messages=messages,
               temperature=self.temperature,
               max_tokens=10000,
               stream=True
           )

           for chunk in stream:
               if chunk.choices[0].delta.content is not None:
                   yield chunk.choices[0].delta.content
                   await asyncio.sleep(0.01)  # small delay to control stream rate

       except Exception as e:
           print(f"Exception occurred in stream: {str(e)}")
           yield f"Error: {str(e)}"

In [20]:
country_name = "Colombia" ## specify the country
scenario  = "A tourist enters a café in Punta Cana and orders some breakfast and coffee."  ## specify the scenario

In [49]:
scenario_path = "./data/scenarios/hotel.txt"
with open(scenario_path, 'r') as file:
    scenario = file.read()
print(scenario)

A guest arrives at a hotel and interacts with the receptionist to check in.
The receptionist welcomes the guest and confirms their reservation.
They provide details about the room, including its type and amenities (a suite with a sea view and a double bed).
The guest asks if there are any discounts for frequent members, but the receptionist explains that no discounts are available.
However, they offer a late check-out as a courtesy.

The receptionist then hands over the room key and asks about the guest's luggage.
Upon confirming that the guest has two large suitcases, they call a porter to assist with carrying the luggage to the room.
The conversation concludes with the receptionist offering assistance if needed and warmly welcoming the guest to enjoy their stay.


In [ ]:
print(config.PROMPT_TEMPLATE_PATH)
prompt_handler = PromptHandler(config.PROMPT_TEMPLATE_PATH)
prompt = prompt_handler.format_prompt_scenario(country_name, scenario)

In [1]:

# For prompts that only require scenarios (no country specification)
print(config.PROMPT_TEMPLATE_SCENARIO_ONLY_PATH)
prompt_handler = PromptHandler(config.PROMPT_TEMPLATE_SCENARIO_ONLY_PATH)
prompt = prompt_handler.format_prompt_scenario(scenario)

NameError: name 'config' is not defined

In [51]:
print(prompt)

You are an expert content creator for language lessons in Spanish. Your task is to generate a realistic conversation in the form of a transcript.
The main character is the language learner. Make it as realistic as possible.
Only include diaglogue and no narration or context setting.
Use following contextual parameters:
**Specific scenario**
A guest arrives at a hotel and interacts with the receptionist to check in.
The receptionist welcomes the guest and confirms their reservation.
They provide details about the room, including its type and amenities (a suite with a sea view and a double bed).
The guest asks if there are any discounts for frequent members, but the receptionist explains that no discounts are available.
However, they offer a late check-out as a courtesy.

The receptionist then hands over the room key and asks about the guest's luggage.
Upon confirming that the guest has two large suitcases, they call a porter to assist with carrying the luggage to the room.
The conversat

In [40]:
llm_adapter = LLMAdapter(model_name=config.MODEL_NAME)

## Synchronous Output

In [47]:
response = llm_adapter.generate_response(prompt)
print(response)

Language Learner (LL): ¿Tiene usted la hora, por favor? (Do you have the time, please?)

Stranger (S): Sí, es 16:50. (Yes, it's 4:50 PM.)

LL: ¡Gracias! ¿Me podría decir dónde está la estación de tren de Madrid? (Thank you! Could you tell me where the Madrid train station is?)

S: Claro, sigue la callePríncipe de Vergara hasta el final, y por allí verás la estación de tren de Madrid. (Sure, follow Príncipe de Vergara street to the end, and there you will see the Madrid train station.)

LL: ¡Muchas gracias por su ayuda! (Thank you very much for your help!)

S: De nada, buena suerte en tu viaje. (You're welcome, good luck on your journey.)

LL: Necesito llegar antes de las 17:30. (I need to get there before 5:30 PM.)

S: ¡Espero que logres llegar a tiempo! (I hope you make it on time!)

LL: ¡Claro! Me despido. (Of course! Goodbye.)

S: Adiós. (Goodbye.)


## Asynchronous Output

In [52]:
async def stream_convo():
    async def convo_stream():
        async for token in llm_adapter.generate_response_stream(prompt):
            yield token
    async for token in convo_stream():
        print(token, end='', flush=True)

In [53]:
await stream_convo()

 Receptionist: ¡Bienvenido! ¿Cómo estás? (Welcome! How are you?)

Guest: Hola, estoy bien, gracias. ¿Soy el señor...? (Hi, I'm fine, thank you. Am I Mr...?)

Receptionist: ¡Claro! Estás aquí por reserva 345, sí? (Yes, you are here for reservation 345, right?)

Guest: Sí, eso es. ¿En qué tipo de habitación me he reservado? (Yes, that's correct. In what type of room have I reserved?)

Receptionist: Te hemos asignado una suite con vista al mar y una cama matrimonial. (We have assigned you a suite with a sea view and a double bed.)

Guest: ¡Muy bien! ¿Hay algún descuento para miembros frecuentes? (Very good! Is there any discount for frequent members?)

Receptionist: Lo siento, no hay descuentos disponibles en este momento. Sin embargo, podemos ofrecerte una salida tardía como gesto de cortesía. (I'm sorry, there are no discounts available at the moment. However, we can offer you a late check-out as a courtesy.)

Guest: Gracias, eso es muy amable. ¿Tuve que pagar por esto adicionalmente? (